In [1]:

import threading
import math
from keras.models import Sequential
from keras.layers import Dense
import pygad.kerasga
import time
import sys
import random
import string
import socket
import tensorflow as tf
from tensorflow import keras
import sys
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, losses
import keras
from keras.datasets import mnist
from keras.optimizers import Adam
import os 
malicious=[]
# Insert code at beginning of script to load score from backup
dir_path = 'C:/Users/ASAU/OneDrive/Desktop'
back_up = os.path.join(dir_path, 'backup.txt')
models = os.path.join(dir_path, 'models.txt')
#because of multiclass datasets
from keras.utils.np_utils import to_categorical 
import random
(x_train, y_train), (x_test, y_test) = mnist.load_data()
assert(x_train.shape[0] == y_train.shape[0]), "The number of images is not equal .."
assert(x_test.shape[0] == y_test.shape[0]), "The number of images is not equal .."
assert(x_train.shape[1:] == (28, 28)), "The dimension of the images are not 28x28"
assert(x_test.shape[1:] == (28, 28))
num_of_samples = []

cols = 5
num_of_classes = 10

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10) 
x_train = x_train/255 
x_test = x_test/255
num_pixels = 784
x_train = x_train.reshape(x_train.shape[0],
                         num_pixels)
x_test = x_test.reshape(x_test.shape[0],
                         num_pixels)
# print(x_train.shape)
def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim = num_pixels,
                    activation = 'relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(num_of_classes, activation='softmax'))
    model.compile(Adam(lr=0.01),
                    loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model           
     

globalmodel = create_model()
# print(model.summary())
# history = model.fit(x_train, y_train, validation_split=0.1,
#          epochs=10, batch_size=200, verbose=1, shuffle=1)

score = globalmodel.evaluate(x_test, y_test, verbose=0)
# print(type(score))
# print('Test Score:', score[0])
# print('Test Accuracy:', score[1])

class SocketThread(threading.Thread):

    def __init__(self, buffer_size, port,i):
        threading.Thread.__init__(self)
        self.buffer_size = buffer_size
        self.port=port
        self.soc = socket.socket(family=socket.AF_INET, type=socket.SOCK_STREAM)
        self.soc.bind(("192.168.56.1" , self.port))
        # print("Socket is bound to an address & port number.")
        self.soc.listen(1)
        self.connection, client_info = self.soc.accept()
        # print("Running a Thread for the Connection with client")
        # print(self.port)
        self.std=std
        # self.lg=lg
        # self.m=m
        self.i=i
        self.new_weights=[]
    
        

    def run(self):
       
        

        # This while loop allows the server to wait for the client to send data more than once within the same connection.
       
        received_data = self.recv()
        self.reply(received_data)
            

            # print(received_data)
            

    def recv(self):
        # print("nnnn")
        received_data = b""
        while True:
            try:
                data = self.connection.recv(self.buffer_size)
                received_data = data
                
                if len(received_data) > 0:
                    # print('hnn')
                    try:
                        # Decoding the data (bytes).
                        received_data = pickle.loads(received_data)
                        # Returning the decoded data.
                        return received_data

                    except BaseException as e:
                        print("Error Decoding the Client's Data:")
                        return


            except BaseException as e:
                print("Error Receiving Data from the Client: ")
                return


    def reply(self, received_data):
        global keras_ga,data_inputs, data_outputs,globalmodel
        if received_data is None:
            print("nothong received from client")
        else:
            score = globalmodel.evaluate(x_test, y_test, verbose=0)
            if score[1] == 1.0:
                print("model is updated correctly")
                self.soc.close()
                return
            else:
                try:
                    response = pickle.dumps(globalmodel)
                    self.model_averaging(globalmodel, received_data)
                except BaseException as e:
                    print("Error Encoding the Message: {msg}.\n".format(msg=e))
        score = globalmodel.evaluate(x_test, y_test, verbose=0)
        # print('Test Score:', score[0])

    

    def model_averaging(self, model, received):
        model_weights_vector = pygad.kerasga.model_weights_as_vector(model=model)
        model_weights_matrix = pygad.kerasga.model_weights_as_matrix(model=model,
                                                                     weights_vector=model_weights_vector)
        best_model_weights_vector = pygad.kerasga.model_weights_as_vector(model=received)
        best_model_weights_matrix = pygad.kerasga.model_weights_as_matrix(model=received,weights_vector=best_model_weights_vector)
       
        
        new_weights = np.array(len(model_weights_matrix))
        best=np.array(len(best_model_weights_matrix))
        for i in range(0,len((model_weights_matrix))):
            new_weights=np.append(new_weights,model_weights_matrix[i])
        for i in range(0,len((best_model_weights_matrix))):
            best=np.append(best,best_model_weights_matrix[i])
        try:
            new_wei = model_weights_matrix
            for idx, arr in enumerate(new_wei):
                new_wei[idx] = new_wei[idx] + best_model_weights_matrix[idx]
                new_wei[idx] = new_wei[idx] / 2
        except:
            malicious.append(self.i)
            self.soc.close()
            lock = threading.Lock()
            lock.acquire()
        

        W=np.array(len(new_wei))
        for i in range(0,len((new_wei))):
            W=np.append(W,new_wei[i])
    
        dist=np.linalg.norm(W-new_weights)
        # print("distance:" ,dist)
        distance=str(dist)
        with open(back_up, 'w') as w:
            w.write(distance)
  
    def distancex(self):
        # x=pickle.load(open("save.p","rb"))
        if os.path.isfile(back_up):
            with open(back_up, 'r') as r:
                score = r.read()
                return score
    def full_aggregation(self,b):
        if b:
            try:
                if os.path.isfile(models):
                    with open(models, 'r') as r:
                        
                        new_weights = float([line.rstrip('\n') for line in r])
                
                        globalmodel.set_weights(weights=new_weights)
                        return True
            except:
                pass

        
port= 5680
m=100
lg=0
std=[]
listenThread = SocketThread(buffer_size=1024*1024*1024,
                                port=5680,i=1)
listenThread.start()
d1=listenThread.distancex()
d1=float(d1)
listenThread2 = SocketThread(buffer_size=1024*1024*1024,
                                port=5681,i=2)
listenThread2.start()
d2=listenThread2.distancex()
d2=float(d2)
listenThread3 = SocketThread(buffer_size=1024*1024*1024,
                                port=5682,i=3)
    # port+=1
listenThread3.start()
d3=listenThread3.distancex()
d3=float(d3)

listenThread4 = SocketThread(buffer_size=1024*1024*1024,
                                port=5683,i=4)
    # port+=1
listenThread4.start()
d4=listenThread4.distancex()
d4=float(d4)
listenThread5 = SocketThread(buffer_size=1024*1024*1024,
                                port=5684,i=5)
    # port+=1
listenThread5.start()
d5=listenThread5.distancex()
d5=float(d5)
listenThread6 = SocketThread(buffer_size=1024*1024*1024,
                                port=5685,i=6)
    # port+=1
# listenThread6.start()
# d6=listenThread6.distancex()
# d6=float(d6)

# listenThread7 = SocketThread(buffer_size=1024*1024*1024,
#                                 port=5686,i=7)
# listenThread7.start()
# d7=listenThread7.distancex()
# d7=float(d7)
# listenThread8 = SocketThread(buffer_size=1024*1024*1024,
#                                 port=5687,i=8)
# listenThread8.start()
# d8=listenThread8.distancex()
# d8=float(d8)

std=np.append(std,d1)
std=np.append(std,d2)
std=np.append(std,d3)
std=np.append(std,d4)
std=np.append(std,d5)
# std=np.append(std,d6)
# std=np.append(std,d7)
# std=np.append(std,d8)
print(std)
mean=np.mean(std)
print(mean)
stdx=np.std(std,axis=0)
print("std",stdx)

if d1 > mean+stdx or d1 < mean-stdx or d1==0:
    malicious.append(1)
    b=False

else:
    b=True
    listenThread.full_aggregation(b)
if d2 > mean+stdx or d2 < mean-stdx or d2==0:
    malicious.append(2)
    b=False

else:
    b=True
    listenThread2.full_aggregation(b)
if d3 > mean+stdx or d3 < mean-stdx or d3==0:
    malicious.append(3)
    b=False

else:
    b=True
    listenThread3.full_aggregation(b)
if d4 > mean+stdx or d4 < mean-stdx or d4==0:
    malicious.append(4)
    b=False

else:
    b=True
    listenThread3.full_aggregation(b)

if d5 > mean+stdx or d5 < mean-stdx or d5==0:
    malicious.append(5)
    b=False
else:
    b=True
    listenThread3.full_aggregation(b)
# if d6 > mean+stdx or d6 < mean-stdx or d6==0:
#     malicious.append(6)
#     b=False
#     # print("client 6 is malicious outlying distance")
# else:
#     b=True
#     listenThread3.full_aggregation(b)
# if d7 > mean+stdx or d7 < mean-stdx or d7==0:
#     malicious.append(7)
#     b=False
#     print("client 7 is malicious outlying distance")
# else:
#     b=True
#     listenThread3.full_aggregation(b)
# if d8 > mean+stdx or d8 < mean-stdx or d8==0:
#     malicious.append(8)
#     b=False
#     # print("client 8 is malicious outlying distance")
# else:
#     b=True
#     listenThread3.full_aggregation(b)
for v in malicious:
    print("client",v,"is malicious")


C:\Users\ASAU\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


nothong received from client
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-01-19 04:43:40         1408
metadata.json                                  2023-01-19 04:43:40           64
variables.h5                                   2023-01-19 04:43:40        10832
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...vars
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dense_3
......vars
.........0
.........1
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...vars
Keras model archive saving